## Test on one video

In [1]:
import numpy as np
import pickle
import os
from PIL import Image
import time
from tqdm import tqdm
import shutil
from random import randint
import argparse
import json

import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

from network import *
import subprocess


In [78]:
video_file_path = '/home/lin/UCF-101/Biking/v_Biking_g06_c02.avi' # specific video
dst_directory_path = '/home/lin/test2/' # path to extract
testpath = '/mnt/act/jpegs_256/v_YoYo_g25_c05/'

In [79]:
cmd = 'ffmpeg -i \"{}\" -vf scale=-1:240 \"{}/image_%05d.jpg\"'.format(video_file_path, dst_directory_path)
subprocess.call(cmd, shell=True)
# if this cell outputs 0, this process is successful.

0

In [80]:
class spatial_dataset():
    def __init__(self,root_dir,transform=None):

        self.keys = os.listdir(root_dir)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.keys)

    def load_ucf_image(self,index):
        path = self.root_dir + self.keys[index]
        img = Image.open(path)
        transformed_img = self.transform(img)
        img.close()

        return transformed_img

    def __getitem__(self, idx):
        data = self.load_ucf_image(idx)
        sample = (self.keys[idx],data)
        return sample


In [81]:
transform = transforms.Compose([
                transforms.Scale([224,224]),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
                ])

In [82]:
validation_set = spatial_dataset(root_dir=dst_directory_path, transform =transform)

In [83]:
val_loader = DataLoader(
            dataset=validation_set, 
            batch_size=1, 
            shuffle=False,
            num_workers=2)

Modify the **resume** to the pre-trained model file path

In [49]:
resume='/home/lin/model_best.pth.tar'

In [74]:
model = resnet101(pretrained= True, channel=3).cuda()
checkpoint = torch.load(resume)
#model = torch.load(resume)
model.load_state_dict(checkpoint['state_dict'])
#model = nn.DataParallel(model,device_ids=[0,1])
model.eval()

ResNet(
  (conv1_custom): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1),

In [84]:
result =[]
for i,(name,data) in enumerate(val_loader):
    #print i,name
    data_var = Variable(data, volatile=True).cuda(async=True)
    output = model(data_var)
    result.append(output.data.cpu().numpy())

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


In [67]:
result[0].shape

(1, 101)

In [85]:
array = np.array(result)

predicts = np.argmax(array,axis=2).flatten()

predicts.shape

predictslist = predicts.tolist()

In [63]:
ucf_dict = {1: 'ApplyEyeMakeup', 2: 'ApplyLipstick', 3: 'Archery', 4: 'BabyCrawling', 5: 'BalanceBeam', 6: 'BandMarching', 7: 'BaseballPitch', 8: 'Basketball', 9: 'BasketballDunk', 10: 'BenchPress', 11: 'Biking', 12: 'Billiards', 13: 'BlowDryHair', 14: 'BlowingCandles', 15: 'BodyWeightSquats', 16: 'Bowling', 17: 'BoxingPunchingBag', 18: 'BoxingSpeedBag', 19: 'BreastStroke', 20: 'BrushingTeeth', 21: 'CleanAndJerk', 22: 'CliffDiving', 23: 'CricketBowling', 24: 'CricketShot', 25: 'CuttingInKitchen', 26: 'Diving', 27: 'Drumming', 28: 'Fencing', 29: 'FieldHockeyPenalty', 30: 'FloorGymnastics', 31: 'FrisbeeCatch', 32: 'FrontCrawl', 33: 'GolfSwing', 34: 'Haircut', 35: 'Hammering', 36: 'HammerThrow', 37: 'HandstandPushups', 38: 'HandstandWalking', 39: 'HeadMassage', 40: 'HighJump', 41: 'HorseRace', 42: 'HorseRiding', 43: 'HulaHoop', 44: 'IceDancing', 45: 'JavelinThrow', 46: 'JugglingBalls', 47: 'JumpingJack', 48: 'JumpRope', 49: 'Kayaking', 50: 'Knitting', 51: 'LongJump', 52: 'Lunges', 53: 'MilitaryParade', 54: 'Mixing', 55: 'MoppingFloor', 56: 'Nunchucks', 57: 'ParallelBars', 58: 'PizzaTossing', 59: 'PlayingCello', 60: 'PlayingDaf', 61: 'PlayingDhol', 62: 'PlayingFlute', 63: 'PlayingGuitar', 64: 'PlayingPiano', 65: 'PlayingSitar', 66: 'PlayingTabla', 67: 'PlayingViolin', 68: 'PoleVault', 69: 'PommelHorse', 70: 'PullUps', 71: 'Punch', 72: 'PushUps', 73: 'Rafting', 74: 'RockClimbingIndoor', 75: 'RopeClimbing', 76: 'Rowing', 77: 'SalsaSpin', 78: 'ShavingBeard', 79: 'Shotput', 80: 'SkateBoarding', 81: 'Skiing', 82: 'Skijet', 83: 'SkyDiving', 84: 'SoccerJuggling', 85: 'SoccerPenalty', 86: 'StillRings', 87: 'SumoWrestling', 88: 'Surfing', 89: 'Swing', 90: 'TableTennisShot', 91: 'TaiChi', 92: 'TennisSwing', 93: 'ThrowDiscus', 94: 'TrampolineJumping', 95: 'Typing', 96: 'UnevenBars', 97: 'VolleyballSpiking', 98: 'WalkingWithDog', 99: 'WallPushups', 100: 'WritingOnBoard', 101: 'YoYo'}

The next cell will vote for each class in descending order.

In [86]:
stats = np.zeros(101)
for item in predictslist:
    stats[item] += 1
statsort = np.argsort(-stats)
for i in statsort:
    if stats[i]-0.0 > 1:
        print stats[i],ucf_dict[i+1]
    else:
        break

157.0 Biking
